This time I have decided to make this notebook self-explanatory and publish ready!

# Approach

This time I will not split my training data set into train and valid as I already say that this approach can't score high in Kaggle. The idea this time will be to utilize gridsearch with cross-validation! I must match the score of my R notebook, at least, before I start with Neural Networks in Tensorflow.

In [61]:
import pandas as pd

In [62]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

In [63]:
train.drop(['Name', 'Age', 'Fare', 'Cabin', 'Embarked', 'Ticket'], axis=1, inplace=True)
infer.drop(['Name', 'Age', 'Fare', 'Cabin', 'Embarked', 'Ticket'], axis=1, inplace=True)

In [64]:
df = pd.concat([train, infer], sort=False)

In [65]:
for column in df.columns:
    if column != 'Survived':
        df[column], junk = df[column].factorize()

train = df[df['Survived'].isnull() == False]
infer = df[df['Survived'].isnull() == True]

infer.drop('Survived', axis=1, inplace=True)

In [67]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [68]:
train.columns

Index(['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch'], dtype='object')

In [110]:
estimator = GradientBoostingClassifier()

params = {
    'n_estimators'      : [49],  #[49,50,51,52,53,54]
    'learning_rate'     : [0.1], #[0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13]
    'min_samples_split' : [5],   #[2,3,4,5,6,7,8,9,10]
    'min_samples_leaf'  : [1],   #[1,2,3,4,5,6,7,8,9,10]
    'max_depth'         : [3],   #[3,4,5,6]
}

gsearch = GridSearchCV(estimator, params, cv=5, n_jobs=8, iid=False)

gsearch.fit(train.drop('Survived', axis=1), train['Survived'])

gsearch.best_params_, gsearch.best_score_

({'learning_rate': 0.1,
  'max_depth': 3,
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'n_estimators': 49},
 0.8013724086251905)

In [111]:
predictions = gsearch.best_estimator_.predict(train.drop('Survived', axis=1))

In [112]:
print(classification_report(train['Survived'], predictions))
print(confusion_matrix(train['Survived'], predictions))

             precision    recall  f1-score   support

        0.0       0.81      0.90      0.85       549
        1.0       0.81      0.66      0.72       342

avg / total       0.81      0.81      0.80       891

[[495  54]
 [117 225]]


In [108]:
submission = pd.DataFrame(index=infer.index)

submission['Survived'] = gsearch.best_estimator_.predict(infer)

submission['Survived'] = submission['Survived'].astype('int32')

submission.to_csv('submission-1-baseline.csv')

In [135]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

df = pd.concat([train,infer], sort=False)

In [136]:
df['Age'] = pd.cut(df['Age'], [0,12,19,30,40,50,65,200], labels=False, right=False, include_lowest=True)

In [137]:
df.drop(['Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1, inplace=True)

for column in df.columns:
    if column != 'Survived':
        df[column], junk = df[column].factorize()

In [138]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
1,0.0,0,0,0,0,0
2,1.0,1,1,1,0,0
3,1.0,0,1,0,1,0
4,1.0,1,1,1,0,0
5,0.0,0,0,1,1,0


In [139]:
train = df[df['Survived'].isnull() == False]
infer = df[df['Survived'].isnull() == True]

infer.drop('Survived', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [147]:
estimator = GradientBoostingClassifier()

params = {
    'n_estimators'      : [56,57,58,59,60,61,62,63,64,65],
    'learning_rate'     : [0.08,0.09,0.1,0.11,0.12],
    'min_samples_split' : [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf'  : [1,2,3,4,5,6,7,8,9,10],
    'max_depth'         : [3,4,5,6]
}

gsearch = GridSearchCV(estimator, params, cv=5, n_jobs=8, iid=False)

gsearch.fit(train.drop('Survived', axis=1), train['Survived'])

gsearch.best_params_, gsearch.best_score_

({'learning_rate': 0.12,
  'max_depth': 3,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 59},
 0.8204424305807496)

In [148]:
predictions = gsearch.best_estimator_.predict(train.drop('Survived', axis=1))
print(classification_report(train['Survived'], predictions))
print(confusion_matrix(train['Survived'], predictions))

             precision    recall  f1-score   support

        0.0       0.84      0.91      0.87       549
        1.0       0.83      0.72      0.77       342

avg / total       0.84      0.84      0.83       891

[[500  49]
 [ 97 245]]


In [149]:
submission = pd.DataFrame(index=infer.index)

submission['Survived'] = gsearch.best_estimator_.predict(infer)

submission['Survived'] = submission['Survived'].astype('int32')

submission.to_csv('submission-2-optimal.csv')

In [235]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

df = pd.concat([train,infer], sort=False)

In [236]:
def family_size(row):
    if row['SibSp'] == 0 and row['Parch'] == 0:
        return 0
    else:
        return row['SibSp'] + row['Parch'] + 1

df.loc[:,'Family Size']   = df.apply(lambda x: family_size(x), axis=1)

In [237]:
df['Age'] = pd.cut(df['Age'], [0,12,19,30,40,50,65,200], labels=False, right=False, include_lowest=True)

df.drop(['Name', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'SibSp', 'Parch'], axis=1, inplace=True)

for column in df.columns:
    if column != 'Survived':
        df[column], junk = df[column].factorize()
        
df.head()

,Survived,Pclass,Sex,Age,Family Size
PassengerId,,,,,
1,0.0,0,0,0,0
2,1.0,1,1,1,0
3,1.0,0,1,0,1
4,1.0,1,1,1,0
5,0.0,0,0,1,1


In [238]:
train = df[df['Survived'].isnull() == False]
infer = df[df['Survived'].isnull() == True]

infer.drop('Survived', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [239]:
estimator = GradientBoostingClassifier()

params = {
    'n_estimators'      : [56,57,58,59,60,61,62,63,64,65],
    'learning_rate'     : [0.08,0.09,0.1,0.11,0.12],
    'min_samples_split' : [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf'  : [1,2,3,4,5,6,7,8,9,10],
    'max_depth'         : [3,4,5,6]
}

gsearch = GridSearchCV(estimator, params, cv=5, n_jobs=8, iid=False)

gsearch.fit(train.drop('Survived', axis=1), train['Survived'])

gsearch.best_params_, gsearch.best_score_

({'learning_rate': 0.1,
  'max_depth': 3,
  'min_samples_leaf': 2,
  'min_samples_split': 10,
  'n_estimators': 58},
 0.822677067452258)

In [240]:
predictions = gsearch.best_estimator_.predict(train.drop('Survived', axis=1))
print(classification_report(train['Survived'], predictions))
print(confusion_matrix(train['Survived'], predictions))

             precision    recall  f1-score   support

        0.0       0.84      0.91      0.87       549
        1.0       0.83      0.73      0.78       342

avg / total       0.84      0.84      0.84       891

[[498  51]
 [ 93 249]]


In [241]:
from fancyimpute import MICE

Using TensorFlow backend.


In [296]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

df = pd.concat([train,infer], sort=False)
df['Embarked'].fillna('C', inplace=True)

df1 = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Embarked']]

df1.loc[:,'Sex'], junk = df1.loc[:,'Sex'].factorize()
df1.loc[:,'Ticket'], junk = df1.loc[:,'Ticket'].factorize()
df1.loc[:,'Embarked'], junk = df1.loc[:,'Embarked'].factorize()

matrix = df1.values

imputer = MICE(n_imputations=1000, min_value=0, n_pmm_neighbors=50)
imputed = imputer.complete(matrix)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[MICE] Completing matrix with shape (1309, 7)
[MICE] Starting imputation round 1/1010, elapsed time 0.000
[MICE] Starting imputation round 2/1010, elapsed time 0.001
[MICE] Starting imputation round 3/1010, elapsed time 0.002
[MICE] Starting imputation round 4/1010, elapsed time 0.003
[MICE] Starting imputation round 5/1010, elapsed time 0.004
[MICE] Starting imputation round 6/1010, elapsed time 0.005
[MICE] Starting imputation round 7/1010, elapsed time 0.006
[MICE] Starting imputation round 8/1010, elapsed time 0.007
[MICE] Starting imputation round 9/1010, elapsed time 0.008
[MICE] Starting imputation round 10/1010, elapsed time 0.008
[MICE] Starting imputation round 11/1010, elapsed time 0.009
[MICE] Starting imputation round 12/1010, elapsed time 0.010
[MICE] Starting imputation round 13/1010, elapsed time 0.011
[MICE] Starting imputation round 14/1010, elapsed time 0.012
[MICE] Starting imputation round 15/1010, elapsed time 0.013
[MICE] Starting imputation round 16/1010, elapse

[MICE] Starting imputation round 203/1010, elapsed time 0.180
[MICE] Starting imputation round 204/1010, elapsed time 0.181
[MICE] Starting imputation round 205/1010, elapsed time 0.182
[MICE] Starting imputation round 206/1010, elapsed time 0.183
[MICE] Starting imputation round 207/1010, elapsed time 0.184
[MICE] Starting imputation round 208/1010, elapsed time 0.185
[MICE] Starting imputation round 209/1010, elapsed time 0.186
[MICE] Starting imputation round 210/1010, elapsed time 0.187
[MICE] Starting imputation round 211/1010, elapsed time 0.188
[MICE] Starting imputation round 212/1010, elapsed time 0.188
[MICE] Starting imputation round 213/1010, elapsed time 0.189
[MICE] Starting imputation round 214/1010, elapsed time 0.190
[MICE] Starting imputation round 215/1010, elapsed time 0.191
[MICE] Starting imputation round 216/1010, elapsed time 0.192
[MICE] Starting imputation round 217/1010, elapsed time 0.193
[MICE] Starting imputation round 218/1010, elapsed time 0.194
[MICE] S

[MICE] Starting imputation round 408/1010, elapsed time 0.367
[MICE] Starting imputation round 409/1010, elapsed time 0.368
[MICE] Starting imputation round 410/1010, elapsed time 0.369
[MICE] Starting imputation round 411/1010, elapsed time 0.370
[MICE] Starting imputation round 412/1010, elapsed time 0.371
[MICE] Starting imputation round 413/1010, elapsed time 0.372
[MICE] Starting imputation round 414/1010, elapsed time 0.373
[MICE] Starting imputation round 415/1010, elapsed time 0.374
[MICE] Starting imputation round 416/1010, elapsed time 0.375
[MICE] Starting imputation round 417/1010, elapsed time 0.376
[MICE] Starting imputation round 418/1010, elapsed time 0.377
[MICE] Starting imputation round 419/1010, elapsed time 0.378
[MICE] Starting imputation round 420/1010, elapsed time 0.378
[MICE] Starting imputation round 421/1010, elapsed time 0.379
[MICE] Starting imputation round 422/1010, elapsed time 0.380
[MICE] Starting imputation round 423/1010, elapsed time 0.381
[MICE] S

[MICE] Starting imputation round 620/1010, elapsed time 0.555
[MICE] Starting imputation round 621/1010, elapsed time 0.556
[MICE] Starting imputation round 622/1010, elapsed time 0.557
[MICE] Starting imputation round 623/1010, elapsed time 0.558
[MICE] Starting imputation round 624/1010, elapsed time 0.559
[MICE] Starting imputation round 625/1010, elapsed time 0.560
[MICE] Starting imputation round 626/1010, elapsed time 0.561
[MICE] Starting imputation round 627/1010, elapsed time 0.562
[MICE] Starting imputation round 628/1010, elapsed time 0.563
[MICE] Starting imputation round 629/1010, elapsed time 0.564
[MICE] Starting imputation round 630/1010, elapsed time 0.565
[MICE] Starting imputation round 631/1010, elapsed time 0.565
[MICE] Starting imputation round 632/1010, elapsed time 0.566
[MICE] Starting imputation round 633/1010, elapsed time 0.567
[MICE] Starting imputation round 634/1010, elapsed time 0.568
[MICE] Starting imputation round 635/1010, elapsed time 0.569
[MICE] S

[MICE] Starting imputation round 790/1010, elapsed time 0.744
[MICE] Starting imputation round 791/1010, elapsed time 0.746
[MICE] Starting imputation round 792/1010, elapsed time 0.747
[MICE] Starting imputation round 793/1010, elapsed time 0.748
[MICE] Starting imputation round 794/1010, elapsed time 0.749
[MICE] Starting imputation round 795/1010, elapsed time 0.750
[MICE] Starting imputation round 796/1010, elapsed time 0.751
[MICE] Starting imputation round 797/1010, elapsed time 0.753
[MICE] Starting imputation round 798/1010, elapsed time 0.754
[MICE] Starting imputation round 799/1010, elapsed time 0.756
[MICE] Starting imputation round 800/1010, elapsed time 0.757
[MICE] Starting imputation round 801/1010, elapsed time 0.759
[MICE] Starting imputation round 802/1010, elapsed time 0.760
[MICE] Starting imputation round 803/1010, elapsed time 0.761
[MICE] Starting imputation round 804/1010, elapsed time 0.763
[MICE] Starting imputation round 805/1010, elapsed time 0.764
[MICE] S

[MICE] Starting imputation round 945/1010, elapsed time 0.930
[MICE] Starting imputation round 946/1010, elapsed time 0.931
[MICE] Starting imputation round 947/1010, elapsed time 0.932
[MICE] Starting imputation round 948/1010, elapsed time 0.933
[MICE] Starting imputation round 949/1010, elapsed time 0.935
[MICE] Starting imputation round 950/1010, elapsed time 0.937
[MICE] Starting imputation round 951/1010, elapsed time 0.938
[MICE] Starting imputation round 952/1010, elapsed time 0.939
[MICE] Starting imputation round 953/1010, elapsed time 0.941
[MICE] Starting imputation round 954/1010, elapsed time 0.942
[MICE] Starting imputation round 955/1010, elapsed time 0.943
[MICE] Starting imputation round 956/1010, elapsed time 0.944
[MICE] Starting imputation round 957/1010, elapsed time 0.945
[MICE] Starting imputation round 958/1010, elapsed time 0.946
[MICE] Starting imputation round 959/1010, elapsed time 0.947
[MICE] Starting imputation round 960/1010, elapsed time 0.948
[MICE] S

In [290]:
imputed[:,2]

array([22.        , 38.        , 26.        , ..., 38.5       ,
       33.20185251, 28.80993803])

In [297]:
df.loc[:,'Age'] = imputed[:,2]

In [298]:
df.drop(['Name', 'Fare', 'Cabin', 'Embarked', 'Ticket'], axis=1, inplace=True)

for column in df.columns:
    if column != 'Survived':
        df[column], junk = df[column].factorize()

train = df[df['Survived'].isnull() == False]
infer = df[df['Survived'].isnull() == True]

infer.drop('Survived', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [299]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
1,0.0,0,0,0,0,0
2,1.0,1,1,1,0,0
3,1.0,0,1,2,1,0
4,1.0,1,1,3,0,0
5,0.0,0,0,3,1,0


In [300]:
estimator = GradientBoostingClassifier()

params = {
    'n_estimators'      : [56,57,58,59,60,61,62,63,64,65],
    'learning_rate'     : [0.08,0.09,0.1,0.11,0.12],
    'min_samples_split' : [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf'  : [1,2,3,4,5,6,7,8,9,10],
    'max_depth'         : [3,4,5,6]
}

gsearch = GridSearchCV(estimator, params, cv=5, n_jobs=8, iid=False)

gsearch.fit(train.drop('Survived', axis=1), train['Survived'])

gsearch.best_params_, gsearch.best_score_

({'learning_rate': 0.12,
  'max_depth': 4,
  'min_samples_leaf': 1,
  'min_samples_split': 3,
  'n_estimators': 64},
 0.796928385016315)

In [301]:
predictions = gsearch.best_estimator_.predict(train.drop('Survived', axis=1))
print(classification_report(train['Survived'], predictions))
print(confusion_matrix(train['Survived'], predictions))

             precision    recall  f1-score   support

        0.0       0.86      0.96      0.91       549
        1.0       0.91      0.75      0.83       342

avg / total       0.88      0.88      0.88       891

[[525  24]
 [ 84 258]]


In [302]:
submission = pd.DataFrame(index=infer.index)

submission['Survived'] = gsearch.best_estimator_.predict(infer)

submission['Survived'] = submission['Survived'].astype('int32')

submission.to_csv('submission-3-optimal.csv')

In [303]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

df = pd.concat([train,infer], sort=False)
df['Embarked'].fillna('C', inplace=True)

df1 = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Embarked', 'Survived']]

df1.loc[:,'Sex'], junk = df1.loc[:,'Sex'].factorize()
df1.loc[:,'Ticket'], junk = df1.loc[:,'Ticket'].factorize()
df1.loc[:,'Embarked'], junk = df1.loc[:,'Embarked'].factorize()

matrix = df1.values

imputer = MICE(n_imputations=1000, min_value=0, n_pmm_neighbors=50)
imputed = imputer.complete(matrix)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[MICE] Completing matrix with shape (1309, 8)
[MICE] Starting imputation round 1/1010, elapsed time 0.001
[MICE] Starting imputation round 2/1010, elapsed time 0.003
[MICE] Starting imputation round 3/1010, elapsed time 0.005
[MICE] Starting imputation round 4/1010, elapsed time 0.007
[MICE] Starting imputation round 5/1010, elapsed time 0.008
[MICE] Starting imputation round 6/1010, elapsed time 0.010
[MICE] Starting imputation round 7/1010, elapsed time 0.011
[MICE] Starting imputation round 8/1010, elapsed time 0.013
[MICE] Starting imputation round 9/1010, elapsed time 0.014
[MICE] Starting imputation round 10/1010, elapsed time 0.016
[MICE] Starting imputation round 11/1010, elapsed time 0.018
[MICE] Starting imputation round 12/1010, elapsed time 0.019
[MICE] Starting imputation round 13/1010, elapsed time 0.021
[MICE] Starting imputation round 14/1010, elapsed time 0.022
[MICE] Starting imputation round 15/1010, elapsed time 0.023
[MICE] Starting imputation round 16/1010, elapse

[MICE] Starting imputation round 254/1010, elapsed time 0.374
[MICE] Starting imputation round 255/1010, elapsed time 0.376
[MICE] Starting imputation round 256/1010, elapsed time 0.378
[MICE] Starting imputation round 257/1010, elapsed time 0.379
[MICE] Starting imputation round 258/1010, elapsed time 0.381
[MICE] Starting imputation round 259/1010, elapsed time 0.382
[MICE] Starting imputation round 260/1010, elapsed time 0.384
[MICE] Starting imputation round 261/1010, elapsed time 0.385
[MICE] Starting imputation round 262/1010, elapsed time 0.386
[MICE] Starting imputation round 263/1010, elapsed time 0.388
[MICE] Starting imputation round 264/1010, elapsed time 0.389
[MICE] Starting imputation round 265/1010, elapsed time 0.391
[MICE] Starting imputation round 266/1010, elapsed time 0.392
[MICE] Starting imputation round 267/1010, elapsed time 0.393
[MICE] Starting imputation round 268/1010, elapsed time 0.395
[MICE] Starting imputation round 269/1010, elapsed time 0.396
[MICE] S

[MICE] Starting imputation round 388/1010, elapsed time 0.561
[MICE] Starting imputation round 389/1010, elapsed time 0.563
[MICE] Starting imputation round 390/1010, elapsed time 0.565
[MICE] Starting imputation round 391/1010, elapsed time 0.566
[MICE] Starting imputation round 392/1010, elapsed time 0.568
[MICE] Starting imputation round 393/1010, elapsed time 0.569
[MICE] Starting imputation round 394/1010, elapsed time 0.571
[MICE] Starting imputation round 395/1010, elapsed time 0.572
[MICE] Starting imputation round 396/1010, elapsed time 0.574
[MICE] Starting imputation round 397/1010, elapsed time 0.575
[MICE] Starting imputation round 398/1010, elapsed time 0.577
[MICE] Starting imputation round 399/1010, elapsed time 0.578
[MICE] Starting imputation round 400/1010, elapsed time 0.580
[MICE] Starting imputation round 401/1010, elapsed time 0.582
[MICE] Starting imputation round 402/1010, elapsed time 0.583
[MICE] Starting imputation round 403/1010, elapsed time 0.585
[MICE] S

[MICE] Starting imputation round 521/1010, elapsed time 0.750
[MICE] Starting imputation round 522/1010, elapsed time 0.752
[MICE] Starting imputation round 523/1010, elapsed time 0.753
[MICE] Starting imputation round 524/1010, elapsed time 0.755
[MICE] Starting imputation round 525/1010, elapsed time 0.756
[MICE] Starting imputation round 526/1010, elapsed time 0.758
[MICE] Starting imputation round 527/1010, elapsed time 0.760
[MICE] Starting imputation round 528/1010, elapsed time 0.761
[MICE] Starting imputation round 529/1010, elapsed time 0.763
[MICE] Starting imputation round 530/1010, elapsed time 0.764
[MICE] Starting imputation round 531/1010, elapsed time 0.765
[MICE] Starting imputation round 532/1010, elapsed time 0.767
[MICE] Starting imputation round 533/1010, elapsed time 0.768
[MICE] Starting imputation round 534/1010, elapsed time 0.769
[MICE] Starting imputation round 535/1010, elapsed time 0.771
[MICE] Starting imputation round 536/1010, elapsed time 0.772
[MICE] S

[MICE] Starting imputation round 776/1010, elapsed time 1.125
[MICE] Starting imputation round 777/1010, elapsed time 1.127
[MICE] Starting imputation round 778/1010, elapsed time 1.128
[MICE] Starting imputation round 779/1010, elapsed time 1.130
[MICE] Starting imputation round 780/1010, elapsed time 1.132
[MICE] Starting imputation round 781/1010, elapsed time 1.133
[MICE] Starting imputation round 782/1010, elapsed time 1.134
[MICE] Starting imputation round 783/1010, elapsed time 1.136
[MICE] Starting imputation round 784/1010, elapsed time 1.137
[MICE] Starting imputation round 785/1010, elapsed time 1.139
[MICE] Starting imputation round 786/1010, elapsed time 1.140
[MICE] Starting imputation round 787/1010, elapsed time 1.142
[MICE] Starting imputation round 788/1010, elapsed time 1.143
[MICE] Starting imputation round 789/1010, elapsed time 1.145
[MICE] Starting imputation round 790/1010, elapsed time 1.146
[MICE] Starting imputation round 791/1010, elapsed time 1.147
[MICE] S

[MICE] Starting imputation round 996/1010, elapsed time 1.499
[MICE] Starting imputation round 997/1010, elapsed time 1.501
[MICE] Starting imputation round 998/1010, elapsed time 1.502
[MICE] Starting imputation round 999/1010, elapsed time 1.504
[MICE] Starting imputation round 1000/1010, elapsed time 1.506
[MICE] Starting imputation round 1001/1010, elapsed time 1.507
[MICE] Starting imputation round 1002/1010, elapsed time 1.509
[MICE] Starting imputation round 1003/1010, elapsed time 1.510
[MICE] Starting imputation round 1004/1010, elapsed time 1.511
[MICE] Starting imputation round 1005/1010, elapsed time 1.513
[MICE] Starting imputation round 1006/1010, elapsed time 1.514
[MICE] Starting imputation round 1007/1010, elapsed time 1.516
[MICE] Starting imputation round 1008/1010, elapsed time 1.517
[MICE] Starting imputation round 1009/1010, elapsed time 1.519
[MICE] Starting imputation round 1010/1010, elapsed time 1.520


In [315]:
imputed[:,7]

array([0.        , 1.        , 1.        , ..., 0.40479464, 0.38098511,
       0.44483356])

In [310]:
df.loc[:,'Age'] = imputed[:,2]

df.drop(['Name', 'Fare', 'Cabin', 'Embarked', 'Ticket'], axis=1, inplace=True)

for column in df.columns:
    if column != 'Survived':
        df[column], junk = df[column].factorize()

train = df[df['Survived'].isnull() == False]
infer = df[df['Survived'].isnull() == True]

infer.drop('Survived', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [311]:
predictions = gsearch.best_estimator_.predict(train.drop('Survived', axis=1))
print(classification_report(train['Survived'], predictions))
print(confusion_matrix(train['Survived'], predictions))

             precision    recall  f1-score   support

        0.0       0.86      0.96      0.91       549
        1.0       0.91      0.75      0.83       342

avg / total       0.88      0.88      0.88       891

[[525  24]
 [ 84 258]]


In [316]:
B = pd.DataFrame(index=df.index)

In [317]:
B.loc[:,'Survived'] = imputed[:,7]

In [322]:
C = B.loc[infer.index,:]

In [324]:
C.loc[:,'Survived'] = C.loc[:,'Survived'].apply(lambda x: int(x>0.5))

In [325]:
C.sum()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,0
